In [1]:
import pandas as pd
import sys,codecs
import numpy as np
import re
import os
import matplotlib.pyplot as plt
import scipy.stats as st
import collections
import statsmodels.api as sm
from time import time
import random

In [2]:
import os

NUM_CPU = len(os.sched_getaffinity(0)) #os.cpu_count() 

print(f'CPU count: {NUM_CPU}')

CPU count: 36


In [3]:
NUM_THREADS = 4

os.environ["MKL_NUM_THREADS"]     = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"] = str(NUM_THREADS)
os.environ["OMP_NUM_THREADS"]     = str(NUM_THREADS)

NUM_PROCESS = NUM_CPU // NUM_THREADS
print(f'max process: {NUM_PROCESS}')

max process: 9


In [5]:
# dimension
n = 1200; p = 100; q = 20; d = 3

In [6]:
# true parameter
tau = np.round(np.random.uniform(0.15,0.2,p),4)
rho = np.round(np.random.uniform(0.2,0.9,p),4)
beta0 = np.round(np.random.uniform(0.5,1,(p,10)),4)
beta1 = np.zeros((p,q-10))
beta = np.hstack((beta0, beta1))
bc = np.array(np.hstack([np.random.normal(0,1,(p,d))]))
# Omega mean and variance
mean = np.zeros(p)
ta = 0.15
i, j = np.mgrid[:p, :p]
cov = ta**2*ta**abs(i-j)
cov[list(range(p)),list(range(p))] = ta
sigma2 = np.zeros(p)
for j in range(p):
    sigma2[j] = bc[j,:]@bc[j,:] + ta

In [7]:
# adjacency matrix- LSM
dd = np.random.uniform(0,1,n)
D = -(n/3)*abs(dd.reshape(n,1)@(np.ones(n).reshape(1,n)) - (np.ones(n).reshape(n,1))@dd.reshape(1,n))
P = np.exp(D)/(1+np.exp(D))
P[list(range(n)),list(range(n))] = 0
A = (np.random.binomial(1,P)).reshape((n,n))
ind = (np.where(np.sum(A,1)==0))[0]
for i in ind:
    A[i,random.sample(list(range(n)),6)] = 1
A[list(range(n)),list(range(n))] = 0
W = A/np.sum(A,1).reshape(n,1) 

In [8]:
# check
print(np.sum(A))
np.max(np.sum(A,1))

4919


13

In [135]:
# Save true parameters
pd.DataFrame(rho).to_csv("Results_LSM_BIC/n1200_p100_B100/rho_true_n1200_p100.csv",index=False)
pd.DataFrame(beta).to_csv("Results_LSM_BIC/n1200_p100_B100/beta_true_n1200_p100.csv",index=False)
pd.DataFrame(bc).to_csv("Results_LSM_BIC/n1200_p100_B100/B_true_n1200_p100.csv",index=False)
pd.DataFrame(A).to_csv("Results_LSM_BIC/A_n1500.csv",index=False)

In [10]:
# Data generation function
def data_generator(nn, p, q, d, seed):

    rng = np.random.default_rng(seed) 
    
    q0 = 10
    Omega = rng.multivariate_normal(mean, cov, (nn,), 'raise')   
    X = rng.normal(0,1,(nn,q0))
    Z = rng.normal(0,1,(nn,d))
    
    Y = np.zeros((n,p))
    for j in range(p):
        y_j = np.dot(np.linalg.inv(np.eye(nn)-rho[j]*W), X@beta[j,:q0] + Z@bc[j,:]+Omega[:,j])#
        Y[:,j] = y_j 
        
    return Y,X,Z

In [9]:
# CMLE-gradient
def gardient_initial_lj_0(nn, p, q, WW, para, Yj, XX):
    
    q0 = 10
    
    rho_j = para[0]
    beta_j = para[1:(q0+1)]
    sigma2_j = para[-1]
    
    
    Sj = np.eye(nn)-rho_j*WW
    S_inverse = np.linalg.inv(Sj)
    G = WW@S_inverse
    Ep = Sj@Yj - XX@beta_j
    
    g1 = -np.trace(G)+np.dot((WW@Yj).T,Ep)/sigma2_j
    g2 = (XX.T@Ep)/sigma2_j
    g3 = -nn/(2*sigma2_j)+(Ep.T@Ep)/(2*(sigma2_j**2))
    #print(Ep)
    
    return np.concatenate(([g1],g2,[g3]))

In [10]:
# CMLE-gradient-full q
def gardient_initial_lj(nn, p, q, WW, para, Yj, XX):

    
    rho_j = para[0]
    beta_j = para[1:(q+1)]
    sigma2_j = para[-1]
    
    
    Sj = np.eye(nn)-rho_j*WW
    S_inverse = np.linalg.inv(Sj)
    G = WW@S_inverse
    Ep = Sj@Yj - XX@beta_j
    
    g1 = -np.trace(G)+np.dot((WW@Yj).T,Ep)/sigma2_j
    g2 = (XX.T@Ep)/sigma2_j
    g3 = -nn/(2*sigma2_j)+(Ep.T@Ep)/(2*(sigma2_j**2))
    #print(Ep)
    
    return np.concatenate(([g1],g2,[g3]))

In [11]:
# CMLE -hessian
def hessian_initial_lj_0(nn, p, q, Wt, para, Yj, XX):
    
    q0 = 10
    
    rho_j = para[0]
    beta_j = para[1:(q0+1)]
    sigma2_j = para[-1]
    
    Sj = np.eye(nn)-rho_j*Wt
    S_inverse = np.linalg.inv(Sj)
    G = np.dot(Wt,S_inverse)
    WW = np.dot(Wt.T,Wt)
    WS = np.dot(Wt.T,Sj)
    SS = np.dot(Sj.T,Sj)
    Ep = Sj@Yj - XX@beta_j
    
    
    h11 = -np.trace(G@G)-np.dot(Yj.T@WW,Yj)/sigma2_j
    h12 = h21 = -(XX.T@(Wt@Yj)/sigma2_j).reshape(q0,1)
    h31 = h13 = -np.dot((Wt@Yj).T,Ep)/(sigma2_j**2)
    
    h22 = -XX.T@XX/sigma2_j
    h23 = h32 = -(XX.T@Ep/(2*sigma2_j**2)).reshape(q0,1)
    h33 = nn/(2*sigma2_j**2)-np.dot(Yj.T@SS,Yj)/(sigma2_j**3)
    
    H = np.block([
         [h11, h12.T, h13],
         [h21, h22, h23],
         [h31, h32.T, h33]
    ])
    
    return H

In [12]:
# CMLE-hessian -full q
def hessian_initial_lj(nn, p, q, Wt, para, Yj, XX):
    
    rho_j = para[0]
    beta_j = para[1:(q+1)]
    sigma2_j = para[-1]
    
    Sj = np.eye(nn)-rho_j*Wt
    S_inverse = np.linalg.inv(Sj)
    G = np.dot(Wt,S_inverse)
    WW = np.dot(Wt.T,Wt)
    WS = np.dot(Wt.T,Sj)
    SS = np.dot(Sj.T,Sj)
    Ep = Sj@Yj - XX@beta_j
    
    
    h11 = -np.trace(G@G)-np.dot(Yj.T@WW,Yj)/sigma2_j
    h12 = h21 = -(XX.T@(Wt@Yj)/sigma2_j).reshape(q,1)
    h31 = h13 = -np.dot((Wt@Yj).T,Ep)/(sigma2_j**2)
    
    h22 = -XX.T@XX/sigma2_j
    h23 = h32 = -(XX.T@Ep/(2*sigma2_j**2)).reshape(q,1)
    h33 = nn/(2*sigma2_j**2)-np.dot(Yj.T@SS,Yj)/(sigma2_j**3)
    
    H = np.block([
         [h11, h12.T, h13],
         [h21, h22, h23],
         [h31, h32.T, h33]
    ])
    
    return H

In [13]:
# Newton-CMLE
def newton_sea_initial_0(nn, p, q, Wt, pa0, Yj, XX, max_iter = 50, eps = 1e-4):
    
    q0 = 10
    pa_new = pa0
    for t in range(max_iter):
        pa_pre = pa_new
        gradient = gardient_initial_lj_0(nn, p, q, Wt, pa_pre, Yj, XX)/nn 
        hessian =  hessian_initial_lj_0(nn, p, q, Wt, pa_pre, Yj, XX)/nn 
        diff = np.linalg.solve(hessian+0.001*np.eye(q0+2),gradient)
        pa_new = pa_pre - diff 
        if pa_new[-1]<0.01: pa_new[-1] = 0.01
        if pa_new[0]>1: pa_new[0] = 0.95
        #print(np.max(abs(diff)))
        if np.linalg.norm(diff) < eps:
            break
            
    return pa_new,t+1

In [14]:
# Newton-CMLE-full q
def newton_sea_initial(nn, p, q, Wt, pa0, Yj, XX, max_iter = 100, eps = 1e-4):
    
    pa_new = pa0
    for t in range(max_iter):
        pa_pre = pa_new
        gradient = gardient_initial_lj(nn, p, q, Wt, pa_pre, Yj, XX)/nn 
        hessian =  hessian_initial_lj(nn, p, q, Wt, pa_pre, Yj, XX)/nn 
        diff = np.linalg.solve(hessian+0.001*np.eye(q+2),gradient)
        pa_new = pa_pre - diff 
        if pa_new[-1]<0.1: pa_new[-1] = 0.1
        if pa_new[0]>1: pa_new[0] = 0.95
        # print(np.max(abs(diff)))
        if np.linalg.norm(diff) < eps:
            break
            
    return pa_new,t+1

In [15]:
# SCAD deriative function
def SCAD_deriative_beta(beta_t, lamba, a = 3.7):
    
    abs_beta = np.abs(beta_t)
    grad = np.zeros_like(beta_t)
    
    mask1 = (abs_beta <= lamba)
    mask2 = (abs_beta > lamba) & (abs_beta <= a*lamba)
    
    grad[mask1] = lamba*np.sign(beta_t[mask1])
    grad[mask2] = ((a * lamba - abs_beta[mask2])/(a - 1))*np.sign(beta_t[mask2])
    
    return grad

In [16]:
def newton_sea_SCAD(nn, p, q, Wt, paj, Yj, XX, lamba, a=3.7, max_iter = 50, eps = 1e-3):
    
    rho_j = paj[0]
    beta_new = paj[1:(q+1)]
    sigma2_j = paj[-1]
    for t in range(max_iter):
        beta_pre = beta_new
        Ep = (np.eye(nn)-rho_j*Wt)@Yj - XX@beta_pre
        gradient_beta = -(XX.T@Ep)
        hessian_beta = XX.T@XX
        grad_SCAD = SCAD_deriative_beta(beta_pre, lamba, a)
        S_lam_beta = np.diag(grad_SCAD)/abs(beta_pre)
        diff = np.linalg.inv(hessian_beta + nn*S_lam_beta)@(gradient_beta + nn*S_lam_beta@beta_pre)
        
        beta_new = beta_pre - diff 
        #print(np.max(abs(diff)))
        if np.linalg.norm(diff) < eps:
            break
            
    return beta_new,t+1

In [17]:
def log_likelihood_sar(rho_j, beta_j, sigma2_j, YY, XX, Wt):
    
    nn = len(YY)
    qq = len(beta_j)
    
    A = np.eye(nn) - rho_j * Wt
    det_term = np.log(np.linalg.det(A))  
    residual = YY - rho_j * Wt @ YY - XX @ beta_j
    loglik = - np.log(2 * np.pi * sigma2_j)/2 + det_term/nn - (residual.T @ residual) / (2 * sigma2_j * nn)
    
    return -loglik  # negative log-likelihood

In [18]:
thre = 1e-3
BICn = 100
par = np.zeros(q+2)
par[-1] = 1
sudu = (np.log(q*p)/n)**0.5
lam_set = np.linspace(sudu**9,2*sudu**0.5, BICn)
bic_sh = np.log(n)*(np.log(p*q))/n

In [ ]:
pip install ray

In [ ]:
pip install -U ipywidgets

In [ ]:
# parallel computation framework
import ray

ray.init(num_cpus=NUM_CPU, ignore_reinit_error=True)

In [127]:
# Full process
@ray.remote(num_cpus=4) 
def map_fun_BIC(bb):
    
    Y, X, Z = data_generator(n, p, q, d, seed = bb+166)
    
    BIC_set = np.zeros((BICn,p))
    Ln_j_set = np.zeros((BICn,p))
    theta_ini = np.zeros((p, 2+q))
    for j in range(p):
        ticn1 = time()
        theta_ini[j,:] = newton_sea_initial(n, p, q, W, par, Y[:,j], X)[0]
        rho0_h = theta_ini[j,:][0]
        beta0_h = theta_ini[j,:][1:(q+1)]
        sigma20_h = theta_ini[j,:][-1]
        for b in range(BICn):
            lambda_ = lam_set[b]
            beta_est = newton_sea_SCAD(n, p, q, W, theta_ini[j,:], Y[:,j], X, lambda_)
            beta_scad = np.where(beta_est[0]<thre, 0, beta_est[0])
            Ln_j_set[b,j] = log_likelihood_sar(rho0_h, beta_scad, sigma20_h, Y[:,j], X, W)
            BIC_set[b,j] = Ln_j_set[b,j] + len(np.where(beta_scad!=0)[0])*bic_sh
        tocn1 = time()
        argBIC = np.argmin(BIC_set[:,j])
        print(bb, j, argBIC, tocn1 - ticn1) 
        with open('Results_LSM_BIC/process.txt', 'a') as f1:
            f1.write(str(bb) + ', '+ str(j) + ', '+ str(argBIC) +'\n')
        
    min_index = np.argmin(BIC_set, axis=0)
    beta_estt = np.zeros((p,q))
    for j in range(p):
        lambda_ = lam_set[int(min_index[j])]
        beta_estt[j,:] = newton_sea_SCAD(n, p, q, W, theta_ini[j,:], Y[:,j], X, lambda_)[0]
        
    return BIC_set.T, min_index, beta_estt

In [ ]:
BB = 100
tic1 = time()
tasks = [map_fun_BIC.remote(bb) for bb in range(BB)]
resultsn1200_p1001BIC = ray.get(tasks)  # 等待所有任务完成
toc1 = time()
print(toc1 - tic1) # 总的计算时间

(raylet) Warning: The remote function __main__.map_fun_BIC is very large (11 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(map_fun_BIC pid=56988) 0 0 27 24.93915343284607
(map_fun_BIC pid=56990) 3 0 26 26.857755184173584
(map_fun_BIC pid=56989) 2 0 27 30.945791244506836
(map_fun_BIC pid=56986) 4 0 42 40.365039348602295
(map_fun_BIC pid=56987) 1 0 29 43.37838292121887
(map_fun_BIC pid=56988) 0 1 20 26.627360582351685
(map_fun_BIC pid=56990) 3 1 22 26.509754180908203
(map_fun_BIC pid=56989) 2 1 30 27.52981185913086
(map_fun_BIC pid=56987) 1 1 43 36.785749435424805
(map_fun_BIC pid=56988) 0 2 30 31.72606110572815
(map_fun_BIC pid=56986) 4 1 24 47.979257583618164
(map_fun_BIC pid=56990) 3 2 26 38.540464878082275
(map_fun_BIC pid=56989) 2 2 24 40.55644750595093
(map_fun_BIC pid=56988) 0 3 24 25.580439567565918
(map_fun_BIC pid=56990) 3 3 26 29.81135606765747
(map_fun_BIC p

In [129]:
BIC_set_n1200_p100_B100 = np.zeros((BB,BICn,p))
min_index_n1200_p100_B100 = np.zeros((BB,p))
beta_est_n1200_p100_B100 = np.zeros((BB,p,q))
for bt in range(BB):
    BIC_set_n1200_p100_B100[bt,:,:], min_index_n1200_p100_B100[bt,:], beta_est_n1200_p100_B100[bt,:,:] = resultsn1200_p1001BIC[bt][0].T, resultsn1200_p1001BIC[bt][1], resultsn1200_p1001BIC[bt][2]

In [132]:
# averge selection consistency
1 - np.mean(abs((beta_est_n1200_p100_B100[:,:,:] > thre).astype(int) - (beta > thre).astype(int)))

0.99957

In [131]:
# uniform selection consistency
1 - len(set(np.where((beta_est_n1200_p100_B100[:,:,:] > thre).astype(int) - (beta > thre).astype(int) != 0)[0]))/100

0.65

In [134]:
#Save results
pd.DataFrame(min_index_n1200_p100_B100).to_csv("Results_LSM_BIC/n1200_p100_B100/min_index_LSM_n1200_p100.csv",index=False)
for b in range(BB):
    pd.DataFrame(BIC_set_n1200_p100_B100[b,:,:]).to_csv("Results_LSM_BIC/n1200_p100_B100/BIC_set_LSM_n1200_p100_"+str(b)+'_.csv',index=False)
    pd.DataFrame(beta_est_n1200_p100_B100[b,:,:]).to_csv("Results_LSM_BIC/n1200_p100_B100/beta_est_LSM_n1200_p100_"+str(b)+'_.csv',index=False)